# Canonical MWC model and analytical reductions for the BK ion channel

This file implements two functions that are required as inputs for ```MBAM```:

* ```BK_simulator(phi::AbstractArray, x::AbstractArray; model_id=0)``` 
For an Mx1 vector of parameters ```phi``` and for a DxN array ```x``` containing calcium concentrations and voltages (so D=2) return the Nx1 steady-state probabilities of the channel being open. Different ```model_id```s describe different parameterisations (reduced models) or assays (actual probability or log probability).

* ```BK_reduce(phi::Vector; model_id=0, in_log=1)``` 
For an Mx1 vector of parameters ```phi``` and for a ```model_id``` return the (M-1)x1 vector of parameters to be used in ```BK_simulator``` for the subsequent model. ```in_log``` describes whether the numerical values of ```phi```s are stored and should be returned after a logarithmic transform (used here for numerical stability due to the diverging parameters).



# MBAM model reduction for the MWC model of BK
Model taken from Aldrich 2002 (Equation 1) <br>
Initial parameters taken from Aldrich 2002 and Miranda 2013 PNAS, Suppl fig 5 <br>
For algorithmic details see the ```MBAM``` notebook.

The goal is to find interpretable models with fewer and fewer parameters in each step.
The iterative model reduction with respect to the $P_{open}$ assay proceeds as follows:
+ model 10 (Original model): <br>
$\theta = \{L_0,\ z_L,\ J_0,\ z_J,\ K_d,\ C,\ D,\ E \}$
    * $z_L -> 0$
    
    
+ model 11: <br>
$\phi^1 = \{L_0,\ J_0,\ z_J,\ K_d,\ C,\ D,\ E \}$
    * $L_0^{1/4} \rightarrow 0, \quad C \rightarrow \infty, \quad D \rightarrow \infty, \quad E \rightarrow 0 $
    

+ model 12: <br>
$\phi^2 = \{J_0,\ z_J,\ K_d,\ L_0^{1/4}C,\ L_0^{1/4}D,\ L_0^{-1/4}E \}$ 
    * $L_0^{1/4}C \rightarrow 0, \quad L_0^{-1/4}E \rightarrow \infty $
    
    
+ model 13: <br>
$\phi^3 = \{J_0,\ z_J,\ K_d,\ CE,\ L_0^{1/4}D \}$ 
    * $J_0 \rightarrow 0, \quad L_0^{1/4}D \rightarrow \infty $
 
 
+ model 14: <br>
$\phi^4 = \{J_0L_0^{1/4}D,\ z_J,\ K_d,\ CE  \}$ 
    * $K_d \rightarrow \infty, \quad CE \rightarrow \infty $
 
 
+ model 15: <br>
$\phi^5 = \{J_0L_0^{1/4}D,\ z_J,\ CEK_d^{-1}  \}$ 


<br>

* The $P_{open}(V, [Ca] \ | \ \phi^3)$ function is an excellent approximation of the original (way below experimental error), despite having only 5 parameters. Each of the parameters are meaningful, the first one describing the slope of the voltage sensitivity, the second the middle point of the voltage sensitivity and the third the calcium sensitivity.
* Note that the 4 parameter-model ($\phi^4$) describes both the voltage and the calcium sensitivity curves as two-parameter sigmoids. The allosteric MWC model introduces nonlinear interaction between these two sensitivity curves.
* Even with 3 paremeters ($\phi^5 = \{J_0L_0^{1/4}D,\ z_J,\ CEK_d^{-1}  \}$ ) the model is capable of capturing the main features of the data. In this case each of the parameters are highly meaningful, the first one describing the slope of the voltage sensitivity, the second the middle point of the voltage sensitivity and the third the calcium sensitivity.


<br>
* For the $\log(P_{open})$ assay reduction, see models 51-56

In [1]:
using Physical

function ndgrid{T}(v1::AbstractArray{T}, v2::AbstractArray{T})
    m, n = length(v1), length(v2)
    v1 = reshape(v1, m, 1)
    v2 = reshape(v2, 1, n)
    (repmat(v1, 1, n), repmat(v2, m, 1))
end

"Return open probabilities for the BK ion channel given the [Ca2+] and the voltage"
function BK_simulator(phi::AbstractArray, x::AbstractArray; model_id=0)
    # Simulates the MWC model (Aldrich 2002) open probabilities of BK given input parameters Phi over the given Ca-V grid points
    # model_id=0 is the full model, further models are reduced representations by MBAM
    Temp = 304 * Kelvin
    elec_resp = (Volt*(-1)*e_electron/(k_boltzmann*304*Kelvin));

    Ca = vcat(x[1,:]...)
    V = vcat(x[2,:]...)
    
    Popen = 0;
    
    # ------------------------------------------- # 
    # ----- Base model with 8 parameters -------- #
    # ------------------------------------------- #
    if mod(model_id,10)==0
        #phi = {L0, zL, J0, zJ, Kd, C, D, E}
        L = phi[1]*exp(-phi[2]*V* elec_resp )
        J = phi[3]*exp(-phi[4]*V* elec_resp )
        K = Ca / (phi[5]);
        C = phi[6]
        D = phi[7]
        E = phi[8]
        Popen = ( L .* (1 + K.*C + J.*D + J.*K.*C.*D.*E).^4 ) ./ ( L .* (1+ K.*C + J.*D + J.*K.*C.*D.*E).^4 + (1 + J + K + J.*K.*E).^4); 
        
        # NOTE: Maybe reparametrize with -1+E, -1+D, -1+C to find actual decouplings in log space?
        
        
    end
    
    
    
    # ------------------------------------------- # 
    # -------- MBAM ran wrt. open probs --------- #
    # ------------------------------------------- #
    if model_id==11
        # zL -> 0
        #phi = {L0, J0, zJ, Kd, C, D, E}
        L = phi[1];
        J = phi[2]*exp(-phi[3]*V* elec_resp )
        K = Ca / (phi[4]);
        C = phi[5]
        D = phi[6]
        E = phi[7]
        Popen = ( L .* (1 + K.*C + J.*D + J.*K.*C.*D.*E).^4 ) ./ ( L .* (1+ K.*C + J.*D + J.*K.*C.*D.*E).^4 + (1 + J + K + J.*K.*E).^4);     
    end
    
    if model_id==12
        # Usual L0^1/4 combined with C, D and E
        #phi ={J0, zJ, Kd, L0^1/4 * C, L0^1/4 * D, E / L0^1/4}
        J = phi[1]*exp(-phi[2]*V* elec_resp )
        K = Ca / (phi[3]);
        C = phi[4]
        D = phi[5]
        E = phi[6]
        
        Popen = ( (K.*C + J.*D + J.*K.*C.*D.*E).^4 ) ./ ( (K.*C + J.*D + J.*K.*C.*D.*E).^4 + (1 + J + K).^4);     
  
    end
    
    if model_id==13
        # C*E finite, C -> 0, E -> infinity
        #phi ={J0, zJ, Kd, C * E , L0^1/4 * D}
        J = phi[1]*exp(-phi[2]*V* elec_resp )
        K = Ca / (phi[3]);
        CE = phi[4]
        D = phi[5]
        
        Popen = ( (J.*D + J.*K.*CE.*D).^4 ) ./ ( (J.*D + J.*K.*CE.*D).^4 + (1 + J + K).^4);    
    end
    
    if model_id==14
        # L0^1/4 *J0*D finite, J0->0, L0^1/4 *D->inf
        #phi ={L0^1/4 *J0*D, zJ, Kd, C * E}
        
        JD = phi[1]*exp(-phi[2]*V* elec_resp )
        K = Ca / (phi[3]);
        CE = phi[4]
        
        Popen = ( (JD + JD.*K.*CE).^4 ) ./ ( (JD + JD.*K.*CE).^4 + (1 + K).^4);
        
    end
    
    if model_id==15
        # (C * E)/Kd finite, C*E->Inf, Kd->inf
        #phi ={L0^1/4 *J0*D, zJ, (C * E)/Kd}
        JD = phi[1]*exp(-phi[2]*V* elec_resp )
        KCE = Ca * (phi[3]);
        
        Popen = ( (JD + JD.*KCE).^4 ) ./ ( (JD + JD.*KCE).^4 + 1.0);
    end
        
    
    
    # ------------------------------------------- # 
    # ------- MBAM ran wrt. log open probs ------ #
    # ------------------------------------------- # 
    if model_id==51
        #phi = {L0, zL, J0, zJ, Kd, C, D, E}
        L = phi[1]*exp(-phi[2]*V* elec_resp )
        J = phi[3]*exp(-phi[4]*V* elec_resp )
        K = Ca / (phi[5]);
        C = phi[6]
        D = phi[7]
        E = phi[8]
        Popen = ( L .* (1 + K.*C + J.*D + J.*K.*C.*D.*E).^4 ) ./ ( L .* (1+ K.*C + J.*D + J.*K.*C.*D.*E).^4 + (1 + J + K + J.*K.*E).^4); 
        Popen = log10(Popen)
    end
    
    if model_id==52
        # zL -> 0
        #phi = {L0, J0, zJ, Kd, C, D, E}
        L = phi[1];
        J = phi[2]*exp(-phi[3]*V* elec_resp )
        K = Ca / (phi[4]);
        C = phi[5]
        D = phi[6]
        E = phi[7]
        Popen = ( L .* (1 + K.*C + J.*D + J.*K.*C.*D.*E).^4 ) ./ ( L .* (1+ K.*C + J.*D + J.*K.*C.*D.*E).^4 + (1 + J + K + J.*K.*E).^4);     
        Popen = log10(Popen)
    end
    
    if model_id==53
        # J0 -> 0, D-> Inf
        #phi = {L0, J0*D, zJ, Kd, C, E}
        L = phi[1];
        JD = phi[2]*exp(-phi[3]*V* elec_resp )
        K = Ca / (phi[4]);
        C = phi[5]
        E = phi[6]
        Popen = ( L .* (1 + K.*C + JD + JD.*K.*C.*E).^4 ) ./ ( L .* (1+ K.*C + JD + JD.*K.*C.*E).^4 + (1 + K).^4);     
        Popen = log10(Popen)
    end
    
    if model_id==54
        # Usual L0^1/4 combined with C, D and E
        #phi ={L0^1/4 * D * J0, zJ, Kd, L0^1/4 * C, E / L0^1/4}
        LJD = phi[1]*exp(-phi[2]*V* elec_resp )
        K = Ca / (phi[3]);
        LC = phi[4]
        LE = phi[5]
        Popen = ( (K.*LC + LJD + LJD.*K.*LC.*LE).^4 ) ./ ( ( K.*LC + LJD + LJD.*K.*LC.*LE).^4 + (1 + K).^4);     
        Popen = log10(Popen)
        
    end
    
    if model_id==55
        # LC -> 0, LE -> Inf
        #phi ={L0^1/4 * D * J0, zJ, Kd, L0^1/4 * C, E / L0^1/4}
        LJD = phi[1]*exp(-phi[2]*V* elec_resp )
        K = Ca / (phi[3]);
        CE = phi[4]
        
        Popen = ( (LJD + LJD.*K.*CE).^4 ) ./ ( (LJD + LJD.*K.*CE).^4 + (1 + K).^4);     
        Popen = log10(Popen)
        
    end
    
    if model_id==56
        # LC -> 0, LE -> Inf
        #phi ={L0^1/4 * D * J0, zJ, Kd, L0^1/4 * C, E / L0^1/4}
        LJD = phi[1]*exp(-phi[2]*V* elec_resp )
        CEK = Ca .* (phi[3]);
        
        Popen = ( (LJD + LJD.*CEK).^4 ) ./ ( (LJD + LJD.*CEK).^4 + (1).^4);     
        Popen = log10(Popen)
        
    end
    
    
    # ------------------------------------------- # 
    # ----- Extra models for Appendix fig ------- #
    # ------------------------------------------- # 
    
    # Reduction steps for Figures/saves/figure_A1_20161108T15035400.jld
    if model_id==71
        # L0 -> 0, D-> Inf
        # {L0*D, zL, J0, zJ, Kd, C, E}
        LD = phi[1]
        zL = exp(-phi[2]*V* elec_resp )
        J = phi[3]*exp(-phi[4]*V* elec_resp )
        K = Ca / (phi[5]);
        C = phi[6]
        E = phi[7]
        Popen = ( zL.*(J.*LD + J.*K.*C.*LD.*E).^4 ) ./ ( zL.*(J.*LD + J.*K.*C.*LD.*E).^4 + (1 + J + K + J.*K.*E).^4); 
        
    end
    
    if model_id==72
        # Kd -> Inf, E -> Inf
        # {L0*D, zL, J0, zJ, E/Kd, C}
        LD = phi[1]
        zL = exp(-phi[2]* V* elec_resp )
        J = phi[3]*exp(-phi[4]*V* elec_resp )
        EK = Ca * phi[5];
        C = phi[6]
        Popen = ( zL.*(J.*LD + J.*EK.*C.*LD).^4 ) ./ ( zL.*(J.*LD + J.*EK.*C.*LD).^4 + (1 + J + J.*EK).^4); 
        
    end
    
    return Popen
end

    

BK_simulator (generic function with 1 method)

In [2]:
"Return a reduced parameters vector given a parameter vector that includes diverging dimensions"
function BK_reduce(phi::Vector; model_id=0, in_log=1)
    if in_log==1
        f_add = (a,x,b,y) -> a*x + b*y
    else
        f_add = (a,x,b,y) -> (x.^a)*(y.^b)
    end

    phi_red = zeros(size(phi)[1]-1);
    
    if model_id==0
        phi_red = phi;
    
    # ------------------------------------------- # 
    # ------- MBAM ran in wrt. open probs ------- #
    # ------------------------------------------- # 
    elseif model_id==10
        phi_red[1] = phi[1]
        phi_red[2] = phi[3]
        phi_red[3] = phi[4]
        phi_red[4] = phi[5]
        phi_red[5] = phi[6]
        phi_red[6] = phi[7]
        phi_red[7] = phi[8]
        
    elseif model_id==11
        #phi = {J0, zJ, Kd, L0^1/4 * C, L0^1/4 * D, E / L0^1/4}
        phi_red = zeros(size(phi)[1]-1)
        phi_red[1] = phi[2]
        phi_red[2] = phi[3]
        phi_red[3] = phi[4]
        phi_red[4] = f_add(1/4, phi[1], 1, phi[5])
        phi_red[5] = f_add(1/4, phi[1], 1, phi[6])
        phi_red[6] = f_add(1,phi[7], -1/4, phi[1])
        
    elseif model_id==12
        # C*E finite, C -> 0, E -> infinity
        #phi ={J0, zJ, Kd, C * E , L0^1/4 * D}
        phi_red[1] = phi[1]
        phi_red[2] = phi[2]
        phi_red[3] = phi[3]
        phi_red[4] = f_add(1, phi[4], 1, phi[6])
        phi_red[5] = phi[5]
        
    elseif model_id==13
        # L0^1/4 *J0*D finite, J0->0, L0^1/4 *D->inf
        #phi ={L0^1/4 *J0*D, zJ, Kd, C * E}
        phi_red = zeros(size(phi)[1]-1)
        phi_red[1] = f_add(1, phi[1], 1, phi[5])
        phi_red[2] = phi[2]
        phi_red[3] = phi[3]
        phi_red[4] = phi[4]
        
    elseif model_id==14
        phi_red = zeros(size(phi)[1]-1)
        # (C * E)/Kd finite, C*E->Inf, Kd->inf
        #phi ={L0^1/4 *J0*D, zJ, (C * E)/Kd}
        phi_red[1] = phi[1]
        phi_red[2] = phi[2]
        phi_red[3] = f_add(1, phi[4], -1, phi[3])
        
    
    # ------------------------------------------- # 
    # ----- MBAM ran in wrt. log open probs ----- #
    # ------------------------------------------- # 
    elseif model_id==51
        phi_red[1] = phi[1]
        phi_red[2] = phi[3]
        phi_red[3] = phi[4]
        phi_red[4] = phi[5]
        phi_red[5] = phi[6]
        phi_red[6] = phi[7]
        phi_red[7] = phi[8]
        
        
    elseif model_id==52
        phi_red[1] = phi[1]
        phi_red[2] = f_add(1, phi[2], 1, phi[6])
        phi_red[3] = phi[3]
        phi_red[4] = phi[4]
        phi_red[5] = phi[5]
        phi_red[6] = phi[7]
        
    elseif model_id==53
        phi_red[1] = f_add(1/4, phi[1], 1, phi[2])
        phi_red[2] = phi[3]
        phi_red[3] = phi[4]
        phi_red[4] = f_add(1/4, phi[1], 1, phi[5])
        phi_red[5] = f_add(1,phi[6], -1/4, phi[1])
        
    elseif model_id==54
        phi_red[1] = phi[1]
        phi_red[2] = phi[2]
        phi_red[3] = phi[3]
        phi_red[4] = f_add(1, phi[4], 1, phi[5])
    
    elseif model_id==55
        phi_red[1] = phi[1]
        phi_red[2] = phi[2]
        phi_red[3] = f_add(1, phi[4], -1, phi[3])
        

     # ------------------------------------------- # 
    # ----- Extra models for Appendix fig ------- #
    # ------------------------------------------- # 
    elseif model_id==70
        phi_red[1] = f_add(1/4, phi[1], 1, phi[7])
        phi_red[2] = phi[2]
        phi_red[3] = phi[3]
        phi_red[4] = phi[4]
        phi_red[5] = phi[5]
        phi_red[6] = phi[6]
        phi_red[7] = phi[8]
        
    elseif model_id==71
        phi_red[1] = phi[1]
        phi_red[2] = phi[2]
        phi_red[3] = phi[3]
        phi_red[4] = phi[4]
        phi_red[5] = f_add(-1, phi[5], 1, phi[7])
        phi_red[6] = phi[6]
        
    else
        
        phi_red = phi;
    end
    
    
   
    
    
    
    return phi_red;
end

BK_reduce (generic function with 1 method)